In [15]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
from mittens import GloVe as glove
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
import tensorflow as tf
# from tensorflow.keras.backend import set_session, clear_session, get_session


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [16]:
# Reset Keras Session
def reset_keras(model):
    sess = tf.compat.v1.keras.backend.get_session()
    tf.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name, 
                problem_type, iteration, hidden_unit_size, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

        
    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"
    
    result_path = "results/"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)

In [17]:
def timeseries_model(layer_name, number_of_unit):
    tf.keras.backend.clear_session()
    
    sequence_input = Input(shape=(24,104),  name = "timeseries_input")
    
    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)
    
    logits_regularizer = tf.keras.regularizers.l2(0.01)
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.GlorotUniform(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    model = Model(inputs=sequence_input, outputs=sigmoid_pred)
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [18]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

In [ ]:
epoch_num = 100
model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

unit_sizes = [128, 256]
#unit_sizes = [256]
iter_num = 11
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
layers = ["LSTM", "GRU"]
#layers = ["GRU"]
for each_layer in layers:
    for each_unit_size in unit_sizes:
        for iteration in range(1, iter_num):
            for each_problem in target_problems:
                
                print("Layer: ", each_layer)
                print("Hidden unit: ", each_unit_size)
                print ("Problem type: ", each_problem)
                print("Iteration number: ", iteration)
                print ("__________________")


                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                
                best_model_name = str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                
                checkpoint = ModelCheckpoint(best_model_name, 
                                             monitor='val_loss', 
                                             verbose=0,
                                             save_best_only=True, 
                                             mode='min', 
                                             period=1)


                callbacks = [early_stopping_monitor, checkpoint]

                model = timeseries_model(each_layer, each_unit_size)
                model.fit(x_train_lstm, 
                          y_train[each_problem], 
                          epochs=epoch_num, 
                          verbose=0, 
                          validation_data=(x_dev_lstm, y_dev[each_problem]), 
                          callbacks=callbacks, 
                          batch_size= batch_size)

                model.load_weights(best_model_name)

                probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                                       each_problem, iteration, each_unit_size,type_of_ner)
                reset_keras(model)
                #del model
                tf.keras.backend.clear_session()
                gc.collect()

Layer:  LSTM
Hidden unit:  128
Problem type:  mort_hosp
Iteration number:  1
__________________


2023-04-03 21:54:10.583136: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/lstm/lstm_cell/bias/m/Assign' id:565 op device:{requested: '', assigned: ''} def:{{{node training/Adam/lstm/lstm_cell/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/lstm/lstm_cell/bias/m, training/Adam/lstm/lstm_cell/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-03 21:54:14.961853: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/AddN' id:342 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul, loss/dense/kernel/Regularizer/mul)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will 

0.8738351737139779 0.5594929268985769 0.9147058823529411 0.44313146233382567
Layer:  LSTM
Hidden unit:  128
Problem type:  mort_icu
Iteration number:  1
__________________


2023-04-03 21:54:38.231726: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/lstm/lstm_cell/recurrent_kernel/v/Assign' id:584 op device:{requested: '', assigned: ''} def:{{{node training/Adam/lstm/lstm_cell/recurrent_kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/lstm/lstm_cell/recurrent_kernel/v, training/Adam/lstm/lstm_cell/recurrent_kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-03 21:54:42.363016: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/AddN' id:342 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul, loss/dense/kernel/Regularizer/mul)}}' was changed by setting attribute after it was run by a sessi

0.888961972837303 0.5200978042516514 0.9395927601809955 0.4539877300613497
Layer:  LSTM
Hidden unit:  128
Problem type:  los_3
Iteration number:  1
__________________


...

2023-04-03 21:58:48.372798: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/lstm/lstm_cell/bias/v/Assign' id:589 op device:{requested: '', assigned: ''} def:{{{node training/Adam/lstm/lstm_cell/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/lstm/lstm_cell/bias/v, training/Adam/lstm/lstm_cell/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-03 21:58:53.623565: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/AddN' id:342 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul, loss/dense/kernel/Regularizer/mul)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will 

0.886757955162231 0.4946385705559797 0.9404977375565611 0.386946386946387
Layer:  LSTM
Hidden unit:  128
Problem type:  los_3
Iteration number:  3
__________________


2023-04-03 21:59:19.093783: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/lstm/lstm_cell/recurrent_kernel/m/Assign' id:560 op device:{requested: '', assigned: ''} def:{{{node training/Adam/lstm/lstm_cell/recurrent_kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/lstm/lstm_cell/recurrent_kernel/m, training/Adam/lstm/lstm_cell/recurrent_kernel/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-03 21:59:25.434442: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/AddN' id:342 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul, loss/dense/kernel/Regularizer/mul)}}' was changed by setting attribute after it was run by a sessi

0.6896568161746996 0.6345870659541482 0.6524886877828054 0.5406698564593302
Layer:  LSTM
Hidden unit:  128
Problem type:  los_7
Iteration number:  3
__________________


2023-04-03 21:59:55.534785: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/decay/Assign' id:541 op device:{requested: '', assigned: ''} def:{{{node training/Adam/decay/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/decay, training/Adam/decay/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-03 22:00:00.774746: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/AddN' id:342 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul, loss/dense/kernel/Regularizer/mul)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify node